In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [2]:
# Определение устройства для вычислений (использование GPU, если доступно)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [3]:
model_name_or_path = "ai-forever/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [4]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Создание датасета
train_dataset = TextDataset(tokenizer=tokenizer,file_path="hrd2.txt",block_size=128)

# Создание даталодера (нарезает текст на оптимальные по длине куски)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

C:\Users\Grishin\PycharmProjects\pythonProject\.venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
from transformers import Trainer, TrainingArguments

# Определение аргументов для тренировки модели
training_args = TrainingArguments(
    report_to=None,
    output_dir="finetuned-gpt_ru",
    overwrite_output_dir=True,
    per_device_train_batch_size= 16,
    gradient_accumulation_steps= 4, # копим градиент чтобы использовать больше batch
    max_steps= 1000,
    save_steps= 100,
    eval_steps= 100,
    dataloader_num_workers= 0,
    save_total_limit= 2
)

# Создание тренера для обучения модели
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
)

max_steps is given, it will override any value given in num_train_epochs


In [6]:
# Запуск процесса обучения
trainer.train()

Step,Training Loss
500,3.747400
1000,3.068800


TrainOutput(global_step=1000, training_loss=3.4080736083984373, metrics={'train_runtime': 763.6564, 'train_samples_per_second': 83.807, 'train_steps_per_second': 1.309, 'total_flos': 4144418242560000.0, 'train_loss': 3.4080736083984373, 'epoch': 111.11111111111111})

In [7]:
# Загрузка обученной модели из чекпоинта
model_2 = GPT2LMHeadModel.from_pretrained('finetuned-gpt_ru/checkpoint-1000').to(DEVICE)

In [8]:
# Генерация текста с использованием обученной модели
ques = '</s>'
inputs = tokenizer(ques, return_tensors='pt').to(DEVICE)
generated_token_ids = model_2.generate(
    **inputs,
    temperature=1.2,
    max_new_tokens=128,
    top_p=0.95,
    top_k=10,
    repetition_penalty=1.03,
    length_penalty=1.0,
    no_repeat_ngram_size=2,
    pad_token_id=50257,
)

C:\Users\Grishin\PycharmProjects\pythonProject\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\Grishin\PycharmProjects\pythonProject\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\Grishin\PycharmProjects\pythonProject\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [9]:
# Декодирование и вывод сгенерированного текста
out = tokenizer.decode(generated_token_ids[0][:])
out.split('\n')

['</s>',
 'я не знаю что вы там делали',
 'нууу я не понимаю о чём вы говорите',
 'а чё ты хочешь от меня',
 'давай встретимся и поговорим нормально',
 'ты чё придуриваешься что ли',
 'вы куда звоните то',
 'всё давай давай говори адрес давай',
 'такой вот казус',
 'и что дальше',
 'вот так вот',
 'не понял',
 'это кто говорит',
 'какого военкомата',
 'у вас есть номер телефона',
 'нет я сторож военкомата',
 'что вы хотите',
 'понимаешь это не шутки это реальность',
 'слушай нууу ты же знаешь что эээ',
 'начальнику второго отделения прибыть в триста четырнадцатый кабинет']